<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span><ul class="toc-item"><li><span><a href="#Cleanup-INT-data" data-toc-modified-id="Cleanup-INT-data-4.1">Cleanup INT data</a></span></li><li><span><a href="#cleanup-COVID-TCR-data---from-adaptive" data-toc-modified-id="cleanup-COVID-TCR-data---from-adaptive-4.2">cleanup COVID TCR data - from adaptive</a></span></li><li><span><a href="#Cleanup-VDJdb-file---curated-by-Shirin" data-toc-modified-id="Cleanup-VDJdb-file---curated-by-Shirin-4.3">Cleanup VDJdb file - curated by Shirin</a></span></li></ul></li><li><span><a href="#bind-all" data-toc-modified-id="bind-all-5">bind all</a></span></li><li><span><a href="#Backup" data-toc-modified-id="Backup-6">Backup</a></span></li></ul></div>

# Dependencies

In [1]:
pacman::p_load(data.table)

# Functions

In [2]:
toString_onefle.fx <- function(df, tostringvar){
    setDT(df)
    ab <- df[, .(tostringvar = toString( eval(parse( text = tostringvar))), #bind subjects
                 count = sum(count)),  #get sum of counts
             by = c("CDR3b", "TRBV", "TRBJ")] #get duplicates sequences with the same cdr3 + TRBV + TRBJ
    df_ab <- merge(df, ab, by = c("CDR3b","TRBV","TRBJ")) # merge together
    df_ab_dedup <- dplyr::distinct(df_ab, CDR3b, TRBV, TRBJ, tostringvar, .keep_all= TRUE)
    
    return(df_ab_dedup[, c("CDR3b", "TRBV", "TRBJ", "samplename", 
                           "count.y", "clonefraction", "tostringvar", "file")])
}

In [3]:
#source("/Users/anabbi/git/ped_CapTCRseq/R/Immune_diversity.R")

# Paths

In [5]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [6]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/"

# Main

In [7]:
load(file = paste0(datapath, "capTCRseq/gliph_int_covid_tcr.RData"))

In [8]:
gliphoutput <- read.csv(paste0(h4hpath,'/GLIPH/int_covid_tcr_cluster.csv'),
                       header = T)

In [9]:
head(gliphoutput)

,index,pattern,Fisher_score,number_subject,number_unique_cdr3,final_score,hla_score,vb_score,expansion_score,length_score,⋯,HLA.B,HLA.C,HLA.DPA1,HLA.DPB1,HLA.DQA1,HLA.DQB1,HLA.DRB1,HLA.DRB3,HLA.DRB4,HLA.DRB5
,<int>,<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,GIWV,0,88,82,6.7e-09,1,1,0.003,0.22,⋯,-,-,-,-,-,-,-,-,-,-
2,1,GIWV,0,88,82,6.7e-09,1,1,0.003,0.22,⋯,-,-,-,-,-,-,-,-,-,-
3,1,GIWV,0,88,82,6.7e-09,1,1,0.003,0.22,⋯,-,-,-,-,-,-,-,-,-,-
4,1,GIWV,0,88,82,6.7e-09,1,1,0.003,0.22,⋯,-,-,-,-,-,-,-,-,-,-
5,1,GIWV,0,88,82,6.7e-09,1,1,0.003,0.22,⋯,-,-,-,-,-,-,-,-,-,-
6,1,GIWV,0,88,82,6.7e-09,1,1,0.003,0.22,⋯,-,-,-,-,-,-,-,-,-,-


In [10]:
ls()

[1] "datapath"            "gitpath"             "gliph_int_covid_tcr"
[4] "gliphoutput"         "h4hpath"             "manifestpath"       
[7] "plotpath"